<a href="https://colab.research.google.com/github/mcallara/probabilistic-forecasting-for-planning-101/blob/main/Part_IV_Planning_Decisions%5BPatric_Hammler%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Structure of the Notebook

2. Introduction to the Business Cases [5 min]
3. Simulation framework [3 min]
4. Code Setup [1 min]
5. Environment Modeling [15 min]
6. Agent Modeling [5 min]
7. Build the Simulation Framework [10 min]
8. Decision-Making [5 min]

# 2. Introduction to the Business Case
## 2.1 Perspective
As a data science consultant engaged by a pharmaceutical company, you provide expertise in optimizing inventory decisions. Under the guidance of Stephanie Brueckner, the head of the commercial department, you have access to a dataset comprising 100 potential future demand scenarios.

## 2.2 The Business Case
The client, operating a retail warehouse, seeks to determine optimal reorder quantities to facilitate timely replenishment at a factory. The factory, responsible for forecasting demand to plan production capacities, is initiating communication with all retail warehouses to coordinate reorder decisions. The primary inquiry pertains to the retail warehouse's timing and volume of reorder placements throughout the future. The objective is to identify reorder quantities that minimize operational costs, encompassing holding, shortage, and transportation expenses. Adherence to a predefined reorder policy outlined in the contractual agreement is imperative.

## 2.3 The Reorder Policy
Orders are restricted to once per month and must be submitted within the initial week of each month. Additionally, there exists a 14 days lead time, denoting the duration between order placement and replenishment. The ultimate goal is to determine the optimal reorder quantity for all forthcoming orders, synthesizing available data and pertinent parameters to inform decision-making effectively.

## 2.4 Optimization objective

### 2.4.1 Holding costs
The holding cost results from warehouse operations, which entail expenses such as energy consumption and rental fees, which increase proportionally with inventory levels.

### 2.4.2 Shortage costs
On the other hand, shortage costs arise from lost sales and potential damage to the company's reputation due to stockouts. Given the critical nature of the pharmaceutical product, particularly its impact on patients' lives, the emphasis lies on avoiding stockouts at all costs, thus warranting a high shortage cost factor.

### 2.4.3 Transportation costs
Transportation costs encompass all expenses associated with the movement of inventory from the factory to the retail warehouse, covering logistics and pre-replenishment quality assessments. Notably, these costs exhibit minimal variation with quantity and are consistently estimated at 20000 CHF per shipment.

### 2.4.4 Optimization objective
In our pursuit of optimal reorder quantities, we must strike a delicate balance, considering the trade-off between excess inventory leading to inflated holding costs and insufficient inventory necessitating frequent reorders, which in turn can escalate transportation expenses or result in stockouts. Ultimately, the total cost calculation encompasses the summation of shortage costs, holding costs, and transportation costs.

c_total = c_shortage + c_holding + c_transportation

# 3. Simulation Framework

We aim to address this optimization challenge through the utilization of a simulation framework. The methodology involves defining a reorder policy, comprising a predetermined list of reorder quantities for each forthcoming month, and applying it within an environment designed to closely mimic real-world conditions. The output of this simulation is the incurred cost, reflecting the efficacy of the reorder policy. Leveraging 100 distinct demand trajectories representing various potential future scenarios, we iterate this process for each scenario and subsequently evaluate key performance indicators (KPIs) such as average total cost or maximum cost. We consider these costs as indicative of the quality of a given reorder policy, using them as the focal point of optimization efforts aimed at identifying the policy that minimizes overall costs. For clarity, we delineate between two integral components within the simulation: the environment and the agent.

## 3.1 Environment

The environment serves to depict the dynamics of the system, encapsulating the fluctuating inventory levels within the warehouse at each time step. These variations may arise from factors such as incoming replenishments or outgoing inventory resulting from sales transactions. In the context of this workshop, the environment encompasses both a factory and a retail warehouse. Whenever the retail warehouse initiates a reorder, it triggers the generation of a shipment, which subsequently arrives after a designated number of days, defined as the lead time.

## 3.2 Agent

The agent assumes the responsibility of decision-making within the simulation. In this particular scenario, the agent adheres to a straightforward policy. It evaluates whether the current day falls within the first week of the month. If not, the reorder quantity is set to zero, in accordance with contractual constraints stipulating that only one order per month is permitted. Conversely, if the condition is met, the quantity is selected from a predetermined list of reorder quantities.

# 4. Code Setup

In [ ]:
# import all required packages
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.graph_objects as go
import scipy.optimize as spo
import yaml

In [ ]:
# get the current path
PATH = Path().absolute()

In [ ]:
# download the data
!wget https://raw.githubusercontent.com/mcallara/probabilistic-forecasting-for-planning-101/main/simulations.csv

In [ ]:
# setup a config dictionary - this is important to have a single source of truth
config = {
    'reorder_policy': 'monthly',
    'monthly_reorder_policy': {
        'quantities': [45, 80, 74, 76, 157, 164, 174, 192, 132, 133, 170, 130]
    },
    'retail_warehouse': {
        'start_ioh': 100,
        'sales_price': 100,
        'lost_sales_multiplier': 5,
        'holding_cost_multiplier': 0.1,
        'purchase_price': 80,
        'transportation_cost': 20000,
        'lead_time_in_days': 14
    }
}

In [ ]:
# write a logger class - this is just for the documentation purposes and we can skip it for now
class Logger:
    def __init__(self):
        self.params = {}
        self.policy_params = {}

    def log_values(self, params):
        for key in params:
            if key not in self.params:
                self.params[key] = [params[key]]
            else:
                self.params[key].append(params[key])

    def save(self):
        filename = str(datetime.datetime.now()).replace(" ", "_").replace(":", "-").split(".")[0]
        with open(PATH / "artifacts" / f'{filename}.pickle', 'wb') as handle:
            pickle.dump(self, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def create_log_statistics(self):

        log_statistics = {}

        # calculate the episodic total cost for all realizations
        log_statistics["episodic_cost_distribution"] = [
            sum(wl.params["total_cost"]) for wl in self.params["weekly_logs"]
        ]

        # calculate the mean total cost for all realizations
        log_statistics["episodic_total_cost_mean"] = np.mean(
            [sum(wl.params["total_cost"]) for wl in self.params["weekly_logs"]]
        )

        # calculate the max total cost for all realizations
        log_statistics["episodic_total_cost_max"] = np.max(
            [sum(wl.params["total_cost"]) for wl in self.params["weekly_logs"]]
        )

        # calculate the holding cost ratio compared to total cost for all realizations
        log_statistics["episodic_holding_cost_ratio"] = np.mean(
            [
                sum(wl.params["holding_cost"]) for wl in self.params["weekly_logs"]
            ]
        ) / log_statistics["episodic_total_cost_mean"]

        # calculate the shortage cost ratio compared to total cost for all realizations
        log_statistics["episodic_shortage_cost_ratio"] = np.mean(
            [
                sum(wl.params["shortage_cost"]) for wl in self.params["weekly_logs"]
            ]
        ) / log_statistics["episodic_total_cost_mean"]

        # calculate the transportation cost ratio compared to total cost for all realizations
        log_statistics["episodic_transportation_cost_ratio"] = np.mean(
            [
                sum(wl.params["transportation_cost"]) for wl in self.params["weekly_logs"]
            ]
        ) / log_statistics["episodic_total_cost_mean"]

        # calculate service level
        log_statistics["episodic_service_level_mean"] = np.mean(
            [sum(wl.params["n_served_demand"]) /
             (sum(wl.params["n_served_demand"]) + sum(wl.params["n_unserved_demand"])) for wl in self.params["weekly_logs"]])

        return log_statistics

    def create_episodic_inventory_plot(self):

        fig = go.Figure()
        df = pd.DataFrame()

        inventory_dims = ["n_demand", "n_served_demand", "n_unserved_demand", "n_incoming_supply", "n_inventory_on_hand", "n_ordered"]

        for idx, episode in enumerate(self.params["weekly_logs"]):
            episode.params["episode"] = [idx] * len(episode.params["t"])
            df = pd.concat([df, pd.DataFrame(data=episode.params)])
            fig.add_traces(
                [
                    go.Scatter(visible=idx == 0, line=dict(color="#000000", width=6),
                               name="n served demand", x=episode.params["t"],
                               y=episode.params["n_served_demand"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#0000FF", width=6),
                               name="n unserved demand", x=episode.params["t"],
                               y=episode.params["n_unserved_demand"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#FF0000", width=6),
                               name="n incoming supply", x=episode.params["t"],
                               y=episode.params["n_incoming_supply"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#00FF00", width=6),
                               name="n inventory on hand", x=episode.params["t"],
                               y=episode.params["n_inventory_on_hand"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#00FF00", width=6),
                               name="n demand", x=episode.params["t"],
                               y=episode.params["n_demand"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#00FF00", width=6),
                               name="n ordered", x=episode.params["t"],
                               y=episode.params["n_ordered"])
                ]
            )

        steps = []
        for i in range(len(self.params["weekly_logs"])):
            step = {
                "method": "update",
                "args": [
                    {"visible": [False] * len(fig.data)},
                    {"title": "Inventory levels"}
                ]
            }
            step["args"][0]["visible"][4*i:4*i+len(inventory_dims)] = [True] * len(inventory_dims)
            steps.append(step)

        sliders = [
            {
                "active": 0,
                "currentvalue": {"prefix": "Episode: "},
                "pad": {"t": 50},
                "steps": steps
            }
        ]

        fig.update_layout(sliders=sliders)

        fig.show()

    def create_episodic_cost_plot(self):

        fig = go.Figure()
        df = pd.DataFrame()

        cost_dims = ["total_cost", "holding_cost", "shortage_cost", "transportation_cost"]

        for idx, episode in enumerate(self.params["weekly_logs"]):
            episode.params["episode"] = [idx] * len(episode.params["t"])
            df = pd.concat([df, pd.DataFrame(data=episode.params)])
            fig.add_traces(
                [
                    go.Scatter(visible=idx == 0, line=dict(color="#000000", width=6),
                               name="Total cost [CHF]", x=episode.params["t"],
                               y=episode.params["total_cost"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#0000FF", width=6),
                               name="Holding cost [CHF]", x=episode.params["t"],
                               y=episode.params["holding_cost"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#FF0000", width=6),
                               name="Shortage cost [CHF]", x=episode.params["t"],
                               y=episode.params["shortage_cost"]),
                    go.Scatter(visible=idx == 0, line=dict(color="#00FF00", width=6),
                               name="Transportation cost [CHF]", x=episode.params["t"],
                               y=episode.params["transportation_cost"])
                ]
            )

        steps = []
        for i in range(len(self.params["weekly_logs"])):
            step = {
                "method": "update",
                "args": [
                    {"visible": [False] * len(fig.data)},
                    {"title": "Cost by week"}
                ]
            }
            step["args"][0]["visible"][4*i:4*i+len(cost_dims)] = [True] * len(cost_dims)
            steps.append(step)

        sliders = [
            {
                "active": 0,
                "currentvalue": {"prefix": "Episode: "},
                "pad": {"t": 50},
                "steps": steps
            }
        ]

        fig.update_layout(sliders=sliders)

        fig.show()

    def create_policy_performance_plot(self):

        log_statistics = self.create_log_statistics()
        df = pd.DataFrame(data={"Total cost [CHF]": log_statistics["episodic_cost_distribution"]})
        fig = px.histogram(df, x="Total cost [CHF]", nbins=100)
        fig.add_annotation(
            text=f"Mean cost: {int(log_statistics['episodic_total_cost_mean'] * 10 ** -3)}K CHF <br>"
                 f"Max. cost: {int(log_statistics['episodic_total_cost_max'] * 10 ** -3)}K CHF <br>"
                 f"Holding cost ratio: {round(100 * log_statistics['episodic_holding_cost_ratio'])}% <br>"
                 f"Shortage cost ratio: {round(100 * log_statistics['episodic_shortage_cost_ratio'])}% <br>"
                 f"Transportation cost ratio: {round(100 * log_statistics['episodic_transportation_cost_ratio'])}% <br>"
                 f"Service level: {round(100 * log_statistics['episodic_service_level_mean'])}",
            align='left',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.9,
            y=0.8,
            bordercolor='black',
            borderwidth=1
        )
        fig.show()

# 5. Environment Modeling

**Step 5.1:** Build a Shipment class.

In [ ]:
class Shipment:
    def __init__(self, qty, t, delay=0):
        """Add two class attributes to the constructor. self.quantity reflects the
        number of products included in the shipment. Assign the qty argument to this
        attribute. Furthermore, self.t_arrival reflects the time of arrival at the
        target location. t_arrival is defined as the sum of the current day t (from
        the constructor-argument) and the lead_time_in_days attribute. The
        lead_time_in_days attribute can be accessed with:
        config["retail_warehouse"]["lead_time_in_days"]. Optionally, the
        lead_time_in_days can be increased by a delay variable from the constructor
        argument list. Hint: to add days to a datetime object you can use a
        pd.DateOffset object.
        """

    def has_arrived(self, t_today):
        """Implement a logic, that returns True if t_today is larger or equal (>=)
        the self.t_arrival attribute of the object. False otherwise."""

**Step 5.2:** Build an InventorySystem class.

In [ ]:
class InventorySystem:
    def __init__(self, src_node):
        self.ioh = None
        self.src_node = None

    def reset(self):
        pass

**Step 5.3:** Build a Factory class.

In [ ]:
class Factory(InventorySystem):
    def __init__(self, src_node=None):
        super().__init__(src_node)
        self.src_node = None

    def produce(self, qty, t, delay):
        """Write a produce method. This method takes qty, t, and delay as argument,
        whereas qty is an integer representing the production quantity. t is a datetime
        object representing the current day of simulation, and delay is an integer
        representing the number of days delay of a shipment due to external factors.

        If the production quantity is greater than 0, return a list with one Shipment
        object in it. The Shipment object receives a quantity, the datetime and the delay
        as argument.

        If the production quantity is 0, an empty list shall be returned."""

**Step 5.4:** Build a RetailWarehouse class.

In [ ]:
class RetailWarehouse(InventorySystem):
    def __init__(self, src_node):
        super().__init__(src_node)
        self.ioh = config["retail_warehouse"]["start_ioh"]
        self.shipments = []

    def register_open_order(self, shipment):
        self.shipments += shipment

    def replenish(self, t):
        """Write the replenish method. Initialize a variable named processed as
        an empty list. Initialize a variable named n_incoming as 0. Iterate over
        each shipment in self.shipments with a for loop. Check if the shipment
        has arrived. If it has arrived, increase the n_incoming with the shipment
        quantity and append the shipment to the processed list. After the for loop,
        increase the self.ioh by the number of incoming products. Remove the
        processed shipments from the self.shipments list."""

    def reset(self):
        """After running through one episode (in this case ="one version of the future"
        in terms of demand materialization, the next episode shall start with the same
        start conditions. Thus, set the self.ioh attribute to the start_ioh as done within
        the constructor.)"""

    def sell(self, qty):
        """The retail warehouse sells products directly to the customers. The sell method
        receives the demand quantity as an argument followed by a couple of checks. Firstly,
        it is checked whether the self.ioh is sufficient to fulfill the demand. Then, this
        information is used to generate two new variables: number of sold products (n_sold)
        and number of not_sold products (n_not_sold). The latter is the number of products
        that could have been sold due to the demand - however, were not sold due to insuficient
        inventory. Lastly, the n_sold is used to reduce the self.ioh. This method has n_sold
        and n_not_sold as return variables."""

**Step 5.5:** Build a Environment class.

In [ ]:
class Environment:
    def __init__(self, factory, retail_warehouse, env_args):
        self.args = env_args
        self.factory = factory
        self.retail_warehouse = retail_warehouse

    def step(self, action, demand, t):
        # create new shipment
        delay = 0
        if self.args is not None and len(self.args["lead_time_delay"]) > 0:
            delay = self.args["lead_time_delay"].pop(0)
        shipment = self.factory.produce(action, t, delay)
        self.retail_warehouse.register_open_order(shipment)

        """replenish inventory in retail_warehouse for incoming shipments"""

        """deduct the demand from the retail_warehouse"""

        """calculate the shortage cost. It is defined as the product of n_not_sold
        multiplied with the sales_price and the lost_sales_multiplier. Both factors
        can be accessed from the config["retail_warehouse"] dictionary."""

        """calculate the holding cost. It is defined as the product of the inventory
        on hand in the retail_warehouse and the purchase_price and the holding_cost_multiplier.
        Both factors can be accessed from the config["retail_warehouse"] dictionary."""

        """calculate the transportation cost. This cost is either 0 (when there is no reorder)
        or equals the transportation cost from the config["retail_warehouse"] dictionary."""

        """calculate the total_cost. The total_cost is defined as the sum of holding, shortage
        and transportation cost."""

        # The meta_data can be ignored for now. We use it to log the results which
        # will become relevant later during the analysis.
        meta_data = {
            "total_cost": total_cost,
            "holding_cost": holding_cost,
            "shortage_cost": shortage_cost,
            "transportation_cost": transportation_cost,
            "n_demand": demand,
            "n_ordered": action,
            "n_served_demand": n_sold,
            "n_unserved_demand": n_not_sold,
            "n_incoming_supply": n_incoming,
            "n_inventory_on_hand": self.retail_warehouse.ioh
        }

        # the new state is equal to the inventory on hand level within the retail warehouse.
        new_state = self.retail_warehouse.ioh

        return total_cost, new_state, meta_data

    def reset(self):
        self.retail_warehouse.reset()
        self.factory.reset()
        return self.retail_warehouse.ioh

# 6. Agent Modeling

**Step 6.1:** Build an agent class. The agent interacts with the environment by assessing the current state and making decisions accordingly. These decisions, such as determining the reorder quantity, are then communicated back to the environment for implementation. The agent operates according to a predefined policy, which dictates the relationship between the environment state and the corresponding action taken. In this instance, we solely extract time information from the environment state; however, in more intricate scenarios, we may incorporate additional detailed state information.

In [ ]:
class Agent:
    def __init__(self, policy):
        self.policy = policy

    def select_action(self, state, t):
        return self.policy.select_action(state, t)

**Step 6.2:** Build a MonthlyReorderPolicy class. This class is characterized by a name and a quantities attribute. The quantities represent a list of reorder quantities utilized when within the first week of a month; otherwise, the select_action method returns 0. The get_params method is implemented to retrieve parameters essential for logging purposes.

In [ ]:
class MonthlyReorderPolicy:
    def __init__(self, quantities=None):
        self.name = "monthly_reorder_policy"
        self.quantities = config[self.name]["quantities"] if quantities is None else quantities
        self.counter = 0

    def get_params(self):
        return {"name": self.name, "quantities": config[self.name]["quantities"]}

    def select_action(self, state, t):
        """implement the select_action method. This method returns the reorder quantity.
        The reorder quantity is 0 in the case that the current time t is not within the
        first week of a month. We can check the day_of_month with the t.day attribute.
        The reorder quantity is picked from the self.quantities list whenever t is within
        the first week of a month. We must be sure to pick the right quantity from the
        self.quantities list. Thus, we access the element with the self.counter attribute.
        In the first month, the counter is 0, then it is incremented by 1 everytime when
        t.day is within the first week of the month."""

# 7. Build the Simulation Framework

**Step 7.1**: The simulate function encompasses three main tasks: (1) setting up all logger, warehouses, agents, and env objects; (2) iterating over each demand scenario provided in the simulations.csv dataset; and (3) iterating over each demand value within a given demand scenario.

In [ ]:
def simulate(policy, env_args=None):

    df_full = pd.read_csv("simulations.csv", index_col=0)
    df_full.index = pd.to_datetime(df_full.index)

    """create a factory and a retail_warehouse object with src_node=factory."""

    """create an agent and an env object of class Agent and Environment."""

    episode_logger = Logger()

    # iterate over each demand scenario with a for loop. One column represents
    # one demand scenario.
    for col in df_full.columns:
        df_part = df_full[col]

        """prepare the simulation for the demand scenario.
        - Set total_cost to 0.
        - Get the initial state of the environment with env.reset()
        - Set the policy counter attribute to 0
        - Create a weekly_logger object of class Logger() """

        # iterate over each date within the demand scenario. t equals the date,
        # demand represents the number of product requests for within t.
        for t, demand in df_part.items():

            # constraint: demand is positive and is measured in natural numbers.
            demand = round(max(demand, 0))

            """get the action with the select_action method from the agent."""

            """apply the action to the Environment with the step method. Find the
            required arguments in the step function definition (in the Environment
            class). This method returns cost, new_state and meta_data."""

            """Add the cost from the current time step to the total_cost."""

            # Set the state as the new state, update meta_data and log_values within
            # the weekly_logger object
            state = new_state
            meta_data.update({"t": t, "action": action})
            weekly_logger.log_values(meta_data)

        # Log demand scenario and weekly_logger within the episode_logger object.
        episode_logger.log_values(
            {
                "col": col,
                "weekly_logs": weekly_logger,
            }
        )

    # get, print and return the cost (together with the episode_logger)
    cost = episode_logger.create_log_statistics()["episodic_total_cost_mean"]
    print(cost)

    return cost, episode_logger

# 8. Decision-Making
**Step 8.1:** Explore the agent / environment interaction

In [ ]:
policy = MonthlyReorderPolicy()
policy.quantities = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
_, artifacts = simulate(policy)

In [ ]:
"""Analyze the cost distribution with artifacts.create_policy_performance_plot().
Describe in 3 sentences what you can see as a comment."""

In [ ]:
"""Analyze the inventory levels with artifacts.create_episodic_inventory_plot().
Describe in 3 sentences what you can see as a comment."""

**Step 8.2**: Analyze the plots, try to find reorder quantities that reduce the mean cost.

**Step 11**: Analyze the plots, try to find reorder quantities that reduce the mean cost.

In [ ]:
"""Now, after you have analyzed and documented the consequences of a policy, that
reorders 100 products every month, you can think about how to change the policy,
so that the expected cost go down. How far can you minimize the inventory cost?
- Manipulate the reorder quantities in the policy.quantities list
- Run the simulation
- Create plots as above
- Observe differences and rerun until you minimized the cost"""

policy.quantities = [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

**Step 12:** Apply an optimization algorithm to find the optimal mean cost possible

In [ ]:
# We reduced the expected inventory cost by applying manual changes to the reorder
# quantity list. However, this is time consuming and probably not optimal. Now we are
# curious to find out the mathematically optimal quantities. Thus, we apply an optimization
# algorithm.

# definition of cost function
def fun(quantities):
    policy.quantities = quantities
    cost, _ = simulate(policy)
    return cost

# optimization
res = spo.minimize(fun, x0=policy.quantities, options={"maxiter": 10})
policy.quantities = [int(x) for x in res.x]

"""Question: Which are the optimal reorder quantities? What is the min. archievable cost?"""

In [ ]:
"""Task: Analyze the optimized policy"""